In [1]:
import numpy as np
np.random.seed(13)
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
import math
from __future__ import division
from sklearn.model_selection import KFold
ticker='AAPL'
y=[]
price=pd.read_csv('/home/andrea/Desktop/NLFF/indexesLabeledNew/indexes'+ticker+'.csv')
price=price.iloc[2500:]
#print(price)
trendwindowtime=[3,6]

#'rsi_6'
xtemp=price[['rsi_12']].values
print(xtemp)
#normalizzo
xtemp=xtemp-min(xtemp)/(max(xtemp)-min(xtemp))

xtemp=np.nan_to_num(np.asarray(xtemp, dtype=float))
for t in trendwindowtime:
    #1
#label because of the maket and append values without data
#simo theroy past trend
    x=[]
    y=[]
    print('============================================================')
    print('============================================================')
    print('Working on window:',t)
    for i in range(0,len(price)-t-1):
        y.append(np.sign(price.iloc[i+t+1]['close']-price.iloc[i+1]['open']))
        x.append(xtemp[i])
    y=np.array(y)
    x=np.array(x)
    testlist=[]
    for j in range(0,1):
        permindex=range(0,len(x))
        permindex=np.random.permutation(permindex)
        train=0.8
        nt=math.ceil(len(x)*train)
        trainvalindex=permindex[0:nt]
        testindex=permindex[nt:]
        x_tv=[]
        y_tv=[]
        x_test=[]
        y_test=[]
        x_tv=x[trainvalindex]
        y_tv=y[trainvalindex]
        x_test=x[testindex]
        y_test=y[testindex]
   
        #Fairly sampling the tv
        posindex=np.where( y_tv > 0 )
        negindex=np.where( y_tv < 0 )

        yindex=[]
        nindex=min(len(posindex[0]),len(negindex[0]))

        #for i in range(1,nindex):
        y_tvnew=np.concatenate((y_tv[posindex[0][0:nindex]],y_tv[negindex[0][0:nindex]]))
        x_tvnew=np.concatenate((x_tv[posindex[0][0:nindex]],x_tv[negindex[0][0:nindex]]))

        #Fairly sampling the test 50% 50%
        posindex=np.where( y_test > 0 )
        negindex=np.where( y_test < 0 )

        yindex=[]
        nindex=min(len(posindex[0]),len(negindex[0]))

        #for i in range(1,nindex):
        y_testnew=np.concatenate((y_test[posindex[0][0:nindex]],y_test[negindex[0][0:nindex]]))
        x_testnew=np.concatenate((x_test[posindex[0][0:nindex]],x_test[negindex[0][0:nindex]]))

        nfold=10
        kf = KFold(n_splits=nfold, random_state=13, shuffle=True)


        cspace=np.logspace(-4,4,5)
        gspace=np.logspace(-4,4,5)
        bestsvm=None
        maxacc=0
        cvacc=0
        maxg=0
        maxc=0
        print('Model Selection...')
        #model selection
        for c in cspace:
            for g in gspace:
                cvacc=0
                #crossvalidation
                for train_index, test_index in kf.split(x_tvnew):
                    x_train, x_val = x_tvnew[train_index], x_tvnew[test_index]
                    y_train, y_val = y_tvnew[train_index], y_tvnew[test_index]
                    rbf_svm=svm.SVC(kernel='rbf',C=c,gamma=g)
                    rbf_svm.fit(x_train,y_train)
                    accuracy=rbf_svm.score(x_val,y_val)
                    cvacc=cvacc+accuracy/nfold
                #print('ecco c and gamma',c,g)
                #print(cvacc)
                if(cvacc>maxacc):
                    maxacc=cvacc
                    maxg=g
                    maxc=c
                    bestsvm=rbf_svm

        print('ACCURACY on val:',maxacc)
        print('Gamma:',maxg)
        print('C:',maxc)
        #print(bestsvm)
        rbf_svm=svm.SVC(kernel='rbf',C=maxc,gamma=maxg)
        rbf_svm.fit(x_tvnew,y_tvnew)
        testacc=rbf_svm.score(x_testnew,y_testnew)
        
        testlist.append(testacc)
        print(testacc)
    acc=sum(testlist)/len(testlist)
    print('Accuracy on test set////////////////////////////////////////////////////////////////////////////////')
    print(acc)
    

[[85.50339326]
 [63.38503936]
 [62.43191672]
 ...
 [34.00467172]
 [47.93587088]
 [39.62381054]]
Working on window: 3
Model Selection...
ACCURACY on val: 0.5086124401913875
Gamma: 0.01
C: 100.0
0.5060728744939271
Accuracy on test set////////////////////////////////////////////////////////////////////////////////
0.5060728744939271
Working on window: 6
Model Selection...
ACCURACY on val: 0.5204701492537314
Gamma: 0.01
C: 1.0
0.5122448979591837
Accuracy on test set////////////////////////////////////////////////////////////////////////////////
0.5122448979591837


In [2]:
x=price[['volume','macd','macds', 'boll_ub', 'boll_lb','rsi_6','rsi_12','vr_6_sma','wr_10','wr_6']]
#specifie how long the trend prediction
window=1
#print(x)
for i in range(0,len(price)-window):
    y.append(np.sign(price.iloc[i+window]['close']-price.iloc[i]['open']))



AttributeError: 'numpy.ndarray' object has no attribute 'append'

###### 